<a href="https://colab.research.google.com/github/janaki-sasidhar/url_shortener_python/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Mon May 16 07:17:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]


     |████████████████████████████████| 2.1 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-vwhhdl0z/farm-haystack_fc43f7c3b694489cbbc4f37d355f146c
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-vwhhdl0z/farm-haystack_fc43f7c3b694489cbbc4f37d355f146c
  Resolved https://github.com/deepset-ai/haystack.git to commit 00aa1f41d7c21273d8c312a3fad0b51ddd446672
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB

In [5]:
!apt-get install libmagic-dev
!pip install python-magic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-dev libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 42 not upgraded.
Need to get 332 kB of archives.
After this operation, 5,552 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-dev amd64 1:5.32-2ubuntu0.4 [79.7 kB]
Fetched 332 kB in 1s (353 kB/s)
Selecting previously 

In [6]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [7]:
# In-Memory Document Store
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [8]:
!wget https://gist.githubusercontent.com/janaki-sasidhar/0809bd38d9d1ee3680de2b8780e83582/raw/4a6ca2ac070f600555f05424acc99cabefb389c9/f.py

--2022-05-16 07:27:03--  https://gist.githubusercontent.com/janaki-sasidhar/0809bd38d9d1ee3680de2b8780e83582/raw/4a6ca2ac070f600555f05424acc99cabefb389c9/f.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256028 (250K) [text/plain]
Saving to: ‘f.py’

f.py                100%[===================>] 250.03K  --.-KB/s    in 0.03s   

2022-05-16 07:27:03 (8.14 MB/s) - ‘f.py’ saved [256028/256028]



In [12]:
from f import data
data.keys()

dict_keys([140191, 140192])

In [31]:
# read data from dict and write to text. create folder named property id and move the text to that. 
# !mkdir 140192
# data_140192 = data[140192]
# with open('140192/140192.txt','w') as f:
#   f.write(data_140192)


140192
└── 140192.txt

0 directories, 1 file


In [32]:
docs = convert_files_to_docs(dir_path='140191', clean_func=clean_wiki_text, split_paragraphs=True)


INFO - haystack.utils.preprocessing -  Converting 140191/140191.txt


In [34]:
print(docs)

[<Document: {'content': 'Arizona Goldfield Homeowners Association\nDECLARATION OF COVENANTS, CONDITIONS, RESTRICTIONS\nKAUFMAN AND BROAD OF ARIZONA,INC.\nDECLARATION OF COVENANTS, CONDITIONS,\nKAUFMAN AND BROAD AT ARIZONA GOLDFIELD\nenevecsvsnacesscscvseesessssessesenssccesssensnsesaceusasecsnsasessssesaeeeeeneceassseuentevensaes\nDEFINITIONS1... .e.ccccsccsssecscsecesssscececsseeesnsnesceseesesensneeeenseeensseenes\nPROPERTY AND PERSONS BOUND BYTHIS\nDECLARATION........ccesccssssecescssnsseetssceessessesesseessatereeneesesseeens\nGeneral Declaration...............cccccesscssssseceesesesessseesenrennen\nDeclarant\'s Disclaimer of Representations.............0000..\nEASEMENTS AND RIGHTS OF ENJOYMENTIN THE\nCOMMON AREAG........ccccccccsscceceessncecssseeessesnaeeesseesseeeensseusesssens\nEasements and Rights of Enjoyment..............ccccceseee\nDelegation OfUse...eee csceessesssecnsaeereeserecesesenneeses\nEasement for Common Areas Maintenance.................\nUtility Easements 10.00.0

In [35]:
document_store.write_documents(docs)

In [36]:
# An in-memory TfidfRetriever based on Pandas dataframes

from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO - haystack.nodes.retriever.sparse -  Found 1 candidate paragraphs from 1 docs in DB


In [37]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2


Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [38]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [41]:
prediction = pipe.run(
    query="What are the Leasing Restrictions?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/4 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 4/4 [00:04<00:00,  1.08s/ Batches]


In [45]:
import pprint
# pprint.pprint(prediction)
print_answers(prediction, details="minimum")



Query: What are the Leasing Restrictions?
Answers:
[   {   'answer': 'covenants',
        'context': '. While Declarant\n'
                   'has no reason to believe that any of the restrictive '
                   'covenants contained in this Declaration are\n'
                   'or may be invalid or unenforceable '},
    {   'answer': 'Therestrictions in this Section 5.9 shall be subject to any '
                  'limitations imposed by law',
        'context': 'd level on adjacent properties.\n'
                   'Therestrictions in this Section 5.9 shall be subject to '
                   'any limitations imposed by law.\n'
                   'No antenna, aerial, satellite '},
    {   'answer': 'no\nlarger than five (5) square feet',
        'context': 'quare inches orless; (c) "for sale" and "for lease" signs '
                   'no\n'
                   'larger than five (5) square feet; and (d) signs used by '
                   'Declarant to advertise the Proper'}]
